# 操作资产

在这个笔记本中，你将编程机器人来拼写你的首字母！你需要指定夹爪的起始和目标姿势，以便抓住和放置你的首字母中的字母。你还需要编写一个基于雅可比的速率控制器，并将其连接到你的模拟图中的 `HardwareStation`。

**学习目标：**
1. 执行必要的空间代数来设计抓取和放置姿势
2. 使用差分IK编写夹爪控制器
3. 理解主动控制的IIWA包裹在 `HardwareStation` 中的图结构

**你将构建什么：** 一个IIWA操纵你的首字母的模拟，使它们放置在机器人前面。

**参考：** 结构与[第3章的Pick and Place Demo](https://deepnote.com/workspace/Manipulation-ac8201a1-470a-4c77-afd0-2cc45bc229ff/project/65aad364-ef1c-45f5-a796-fac7c122e274/notebook/pick-d0b41e97e5124292af7ed01072ecece4)非常相似。 <u> 花点时间确保那个笔记本对你有意义。</u>

让我们开始导入并启动Meshcat。

In [9]:
from pathlib import Path

import mpld3
import numpy as np
from pydrake.all import (
    AddFrameTriadIllustration,
    BasicVector,
    Context,
    DiagramBuilder,
    Integrator,
    JacobianWrtVariable,
    LeafSystem,
    MultibodyPlant,
    PiecewisePolynomial,
    PiecewisePose,
    RigidTransform,
    RotationMatrix,
    Simulator,
    StartMeshcat,
    Trajectory,
    TrajectorySource,
)

from manipulation import running_as_notebook
from manipulation.exercises.grader import Grader
from manipulation.exercises.pick.test_pickplace_initials import TestPickPlacePoses
from manipulation.letter_generation import create_sdf_asset_from_letter
from manipulation.station import LoadScenario, MakeHardwareStation

if running_as_notebook:
    mpld3.enable_notebook()

In [10]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7006


抓取姿势通常在对象框架中定义，以便抓取姿势 ${^OX^G}$ 独立于对象的姿势。世界框架中的抓取姿势可以通过以下方式计算：

$${{^WX^G} = {}{^W}X^{O}} {^OX^G},$$

其中 $W$ 代表世界框架，$G$ 表示抓取框架，按照教科书中的约定。

你应该从下面的可视化中注意到，夹爪框架不同于世界框架。特别是，夹爪框架的 +y 轴垂直向下，+z 轴向后。这是这个练习的重要观察。

简单地命令夹爪到抓取姿势并不能确保无碰撞路径。为了解决这个问题，我们定义了一个“预抓取”姿势——一个附近的位置，有足够的间隙让机器人移动到抓取姿势而无碰撞。

**使用图像中的夹爪和对象的方向来设计抓取姿势，使夹爪在对象框架上方0.17米。**

**给定对象姿势X_WO，我们需要X_OG（对象框架中的抓取姿势）和X_WG（世界框架中的抓取姿势）。**

**记住X轴显示为红色，Y轴为绿色，Z轴为蓝色。**

![ps2_fig.png](https://raw.githubusercontent.com/RussTedrake/manipulation/master/book/figures/exercises/grasp_letter.png)

In [11]:
def design_grasp_pose(X_WO: RigidTransform) -> tuple[RigidTransform, RigidTransform]:
    """
    在下面填写我们的代码
    """
    X_OG = RigidTransform()
    X_WG = RigidTransform()
    return X_OG, X_WG

# 设计目标姿势
我们已经确定了机器人夹爪需要移动的位置，给定对象的初始姿势。现在我们需要决定夹爪需要去哪里放置对象。

你会注意到在图中，第一个首字母（"S"）面向正确的方向，但平移偏移（它在机器人后面）。你将为第一个首字母设计一个放置姿势，使其在第二个首字母左侧0.4米（世界框架中的-x）。

第二个首字母（"C"）位于正确的姿势，但旋转使其面向垂直于机器人。我们希望它具有与第一个首字母相同的方向。换句话说，你将为第二个首字母设计一个放置姿势，使其y轴在世界框架中向上，z轴沿世界-y轴。

**编写一个函数，接受你的第一个和第二个首字母的姿势，并计算每个的校正姿势。然后，给定抓取姿势，找到世界框架中实现这些姿势的正确夹爪姿势。**

![ps2_2.png](https://raw.githubusercontent.com/RussTedrake/manipulation/master/book/figures/exercises/place_letter.png)

In [12]:
def design_goal_poses(
    X_WO1: RigidTransform, X_WO2: RigidTransform, X_OG: RigidTransform
) -> tuple[RigidTransform, RigidTransform]:
    """
    在下面填写我们的代码。我们正在为两个对象设计姿势，所以我们定义
    O1为第一个提供的首字母中心的框架，O2为第二个提供的首字母中心的框架。
    """
    X_WG1goal = RigidTransform()
    X_WG2goal = RigidTransform()
    return X_WG1goal, X_WG2goal

# 设计接近姿势

对于抓取和放置姿势，夹爪必须靠近桌子上的字母。如果它从侧面接近那些字母，或者将一个字母移动得太靠近另一个字母，那么碰撞风险会撞倒两个字母中的一个。为了处理这个问题，我们通常为夹爪定义“预抓取”和“预放置”姿势。这些姿势具有机器人可以实现它们而无碰撞风险的特性，并且机器人可以移动到轨迹中的下一个姿势而无不想要的碰撞。

**编写一个函数，接受夹爪姿势X_WG并返回接近姿势X_WGApproach。这个框架从框架G测量时高度为0.1。**

In [13]:
def approach_pose(X_WG: RigidTransform) -> RigidTransform:
    """
    在下面填写我们的代码
    """
    X_WGApproach = RigidTransform()
    return X_WGApproach

# 从关键帧到轨迹

太好了！所以为了操纵两个首字母，机器人将遵循这个轨迹
- 从其默认配置开始（夹爪打开）
- 移动到第一个预抓取姿势（夹爪打开）
- 移动到抓取姿势（夹爪打开）
- 保持在抓取姿势（夹爪关闭）
- 移动到默认配置（夹爪关闭）
- 移动到第一个放置姿势（夹爪关闭）
- 保持在第一个放置姿势（夹爪打开）
- 返回到默认配置并为第二个首字母重复过程

我们已经解决了所有这些姿势！下一步是将它们转换为我们可以命令机器人遵循的有效轨迹。

**你的工作是实现 `MakeTrajectory` 函数。** 它将接受输入
- 长度为L的夹爪姿势列表
- 1xL的numpy数组的夹爪状态（两个手指之间的距离）
- 长度为L的时间戳列表

它应该返回
- 对应于以夹爪姿势定义的分段轨迹的**速度**（类型为 `Trajectory`）
- 定义手指状态的轨迹。

In [14]:
def make_trajectory(
    X_Gs: list[RigidTransform], finger_values: np.ndarray, sample_times: list[float]
) -> tuple[Trajectory, PiecewisePolynomial]:
    robot_velocity_trajectory = None
    traj_wsg_command = None
    # TODO: 从X_Gs定义一个PiecewisePose
    # TODO: 将robot_velocity_trajectory设置为你刚定义的位置轨迹的导数
    # TODO: 将traj_wsg_command设置为命令手指的PiecewisePolynomial
    return robot_velocity_trajectory, traj_wsg_command

# 初始化我们的系统

让我们启动一个包含所有资产的系统。首先，我们将为我们的第一个和最后一个首字母以及桌子生成资产文件。然后，我们以YAML格式定义场景指令。就像问题集1一样！

你会注意到下面调用 `create_sdf_asset_from_letter` 的新关键字。现在我们正在操纵这些资产，而不仅仅是将它们生成到场景中，我们用来表示字母的几何选择会影响我们模拟的物理！通过设置 `use_bbox_collision_geometry=True`，我们选择用它们的_轴对齐边界框_来建模字母。这意味着当MultibodyPlant评估场景的动力学时，它会将字母视为矩形盒子。

**确保填写你的（2）个首字母！**

In [15]:
initials = "LXY"

In [16]:
output_dir = Path("assets/")
for letter in initials:
    create_sdf_asset_from_letter(
        text=letter,
        font_name="DejaVu Sans",
        letter_height_meters=0.2,
        extrusion_depth_meters=0.04,
        output_dir=output_dir / f"{letter}_model",
        use_bbox_collision_geometry=True,
        mass=0.1,
    )
letter_sdfs = [
    f"{Path.cwd()}/assets/{letter}_model/{letter}.sdf" for letter in initials
]

table_sdf = """<?xml version="1.0"?>
<sdf version="1.7">
  <model name="table">
    <pose>0 0 0 0 0 0</pose>
    <link name="table_link">
      <inertial>
        <mass>20</mass>
        <inertia>
          <ixx>1.0</ixx>
          <ixy>0.0</ixy>
          <ixz>0.0</ixz>
          <iyy>1.0</iyy>
          <iyz>0.0</iyz>
          <izz>1.0</izz>
        </inertia>
      </inertial>
      <collision name="box_collision">
        <geometry>
          <box>
            <size>2 2 0.1</size>
          </box>
        </geometry>
      </collision>
      <visual name="box_visual">
        <geometry>
          <box>
            <size>2 2 0.1</size>
          </box>
        </geometry>
      </visual>
    </link>
  </model>
</sdf>
"""

abs_table_sdf_path = "assets/table.sdf"
with open(abs_table_sdf_path, "w") as f:
    f.write(table_sdf)

table_sdf = f"{Path.cwd()}/assets/table.sdf"
scenario_yaml = f"""directives:
- add_model:
    name: iiwa
    file: package://drake_models/iiwa_description/sdf/iiwa7_no_collision.sdf
    default_joint_positions:
        iiwa_joint_1: [-1.57]
        iiwa_joint_2: [0.1]
        iiwa_joint_3: [0]
        iiwa_joint_4: [-1.2]
        iiwa_joint_5: [0]
        iiwa_joint_6: [ 1.6]
        iiwa_joint_7: [0]
- add_weld:
    parent: world
    child: iiwa::iiwa_link_0
- add_model:
    name: wsg
    file: package://manipulation/hydro/schunk_wsg_50_with_tip.sdf
- add_weld:
    parent: iiwa::iiwa_link_7
    child: wsg::body
    X_PC:
        translation: [0, 0, 0.09]
        rotation: !Rpy {{ deg: [90, 0, 90]}}
- add_model:
    name: table
    file: file://{table_sdf}
- add_weld:
    parent: world
    child: table::table_link
    X_PC:
        translation: [0.0, 0.0, -0.05]
        rotation: !Rpy {{ deg: [0, 0, -90] }}
- add_model:
    name: first_initial
    file: file://{letter_sdfs[0]}
    default_free_body_pose:
        {initials[0]}_body_link:
            translation: [0.5, -0.2, 0.01]
            rotation: !Rpy {{ deg: [90, 0, 0] }}
- add_model:
    name: last_initial
    file: file://{letter_sdfs[1]}
    default_free_body_pose:
        {initials[1]}_body_link:
            translation: [-0.2, -0.5, 0.01]
            rotation: !Rpy {{ deg: [90, 0, 90] }}

model_drivers:
    iiwa: !IiwaDriver
      control_mode: position_only
      hand_model_name: wsg
    wsg: !SchunkWsgDriver {{}}
"""

scenario = LoadScenario(data=scenario_yaml)

`triangulate_polygon(*args, engine="triangle")`
to use the non-FSF-approved-license triangle engine
ERROR:root:Error creating SDF asset for letter 'L': No available triangulation engine!
`triangulate_polygon(*args, engine="triangle")`
to use the non-FSF-approved-license triangle engine
ERROR:root:Error creating SDF asset for letter 'X': No available triangulation engine!
ERROR:root:Error creating SDF asset for letter 'L': No available triangulation engine!
`triangulate_polygon(*args, engine="triangle")`
to use the non-FSF-approved-license triangle engine
ERROR:root:Error creating SDF asset for letter 'X': No available triangulation engine!
`triangulate_polygon(*args, engine="triangle")`
to use the non-FSF-approved-license triangle engine
ERROR:root:Error creating SDF asset for letter 'Y': No available triangulation engine!
`triangulate_polygon(*args, engine="triangle")`
to use the non-FSF-approved-license triangle engine
ERROR:root:Error creating SDF asset for letter 'Y': No available 

# 框架选择影响问题陈述！

在上面的图中，注意坐标框架实际上没有放置在字母的中心。相反，它们放置在左下角。相对于这些原点定义抓取姿势很棘手，因为我们希望机器人夹爪在字母上方居中，但字母宽度因不同的首字母而异。在下面的函数中，让 `S` 描述对象框架，因为它在对应字母几何的SDF文件中使用。让 `O` 描述放置在对象质心处的框架。我们上面定义的抓取和放置姿势以框架 `O` 为准，所以让我们确保我们使用的是那个框架。

In [17]:
# 辅助函数，将网格姿势表示为质心而不是几何中心

def get_initial_pose(
    plant: MultibodyPlant, body_name: str, plant_context: Context
) -> RigidTransform:
    body = plant.GetBodyByName(body_name)
    X_WS = plant.EvalBodyPoseInWorld(plant_context, body)
    X_SO = RigidTransform(body.default_spatial_inertia().get_com())
    return X_WS @ X_SO

# 基于雅可比的速度控制器
好的，我们已经指定了场景的几何和我们期望的机器人轨迹。现在我们必须添加逻辑，将我们期望的夹爪速度与我们可以命令的机器人关节速度相关联。

**你的工作是实现CalcOutput函数。回想一下，这接受当前上下文并基于此上下文为 `output` 设置值。**

In [18]:
class PseudoInverseController(LeafSystem):
    def __init__(self, plant: MultibodyPlant):
        LeafSystem.__init__(self)
        self._plant = plant
        self._plant_context = plant.CreateDefaultContext()
        self._iiwa = plant.GetModelInstanceByName("iiwa")
        self._G = plant.GetBodyByName("body").body_frame()
        self._W = plant.world_frame()

        self.V_G_port = self.DeclareVectorInputPort("V_WG", 6)
        self.q_port = self.DeclareVectorInputPort("iiwa.position", 7)
        self.DeclareVectorOutputPort("iiwa.velocity", 7, self.CalcOutput)
        self.iiwa_start = plant.GetJointByName("iiwa_joint_1").velocity_start()
        self.iiwa_end = plant.GetJointByName("iiwa_joint_7").velocity_start()

    def CalcOutput(self, context: Context, output: BasicVector):
        """
        在下面填写我们的代码。
        """
        # 在当前上下文中评估V_G_port和q_port以获取那些值。
        # 根据 `q` 更新内部_plant_context的位置。
        # 提示：你可以通过调用 `self._plant.SetPositions` 写入plant上下文
        # 计算夹爪雅可比
        # 提示：雅可比是6 x N，其中N是DOF的数量。
        # 我们只想要对应IIWA的6 x 7子矩阵
        # 通过将夹爪速度（从V_G_port）映射到关节空间来计算 `v`
        v = None
        output.SetFromVector(v)

# 把一切放在一起

我们快完成了！让我们使用我们定义的场景来初始化图，并指定关键帧！

我们已经添加了图所需的所有系统。**你的工作将是连接所有必要的输入和输出端口以进行模拟。**

In [19]:
# 构建轨迹关键帧
X_OG, X_WG2pick = design_grasp_pose(X_WO2initial)
_, X_WG1pick = design_grasp_pose(X_WO1initial)
X_WG1prepick, X_WG2prepick = approach_pose(X_WG1pick), approach_pose(X_WG2pick)
X_WG1goal, X_WG2goal = design_goal_poses(X_WO1initial, X_WO2initial, X_OG)
X_WG1pregoal, X_WG2pregoal = approach_pose(X_WG1goal), approach_pose(X_WG2goal)

# 手指距离的常量，当夹爪打开或关闭时
opened = 0.107
closed = 0.0

# 关键帧列表，格式为（夹爪姿势，手指状态）
# 对于每个对象，机器人从其默认姿势开始，夹爪打开
# 然后它去到预抓取姿势，抓取姿势，关闭夹爪，然后去到放置姿势
keyframes = [
    (X_WGinitial, opened),
    (X_WG2prepick, opened),
    (X_WG2pick, opened),
    (X_WG2pick, closed),
    (X_WGinitial, closed),
    (X_WG2pregoal, closed),
    (X_WG2goal, closed),
    (X_WG2goal, closed),
    (X_WG2goal, opened),
    (X_WGinitial, opened),
    (X_WG1prepick, opened),
    (X_WG1pick, opened),
    (X_WG1pick, closed),
    (X_WGinitial, closed),
    (X_WG1pregoal, closed),
    (X_WG1goal, closed),
    (X_WG1goal, opened),
    (X_WGinitial, opened),
]

# 解包关键帧并使用它们构建 `Trajectory` 对象
# 注意：我们指定每个关键帧在最后一个之后2秒发生。
gripper_poses = [keyframe[0] for keyframe in keyframes]
finger_states = np.asarray([keyframe[1] for keyframe in keyframes]).reshape(1, -1)
sample_times = [2 * i for i in range(len(gripper_poses))]
traj_V_G, traj_wsg_command = make_trajectory(gripper_poses, finger_states, sample_times)

# V_G_source 定义了夹爪速度的轨迹。将其添加到系统中。
V_G_source = builder.AddSystem(TrajectorySource(traj_V_G))
# 将我们刚定义的DiffIK控制器添加到系统中
controller = builder.AddSystem(PseudoInverseController(plant))
# HardwareStation期望以关节角度形式的机器人命令。
# 我们定义 `integrator` 系统从关节速度映射到关节角度。
integrator = builder.AddSystem(Integrator(7))
# wsg_source 定义了手指位置的轨迹。将其添加到系统中。
wsg_source = builder.AddSystem(TrajectorySource(traj_wsg_command))

# TODO: 将关节速度源连接到伪逆控制器
# TODO: 将控制器连接到积分器以获取关节角度命令
# TODO: 将积分器计算的关节角度连接到操纵站上的iiwa.position端口
# TODO: 将站上的 "iiwa.position_measured" 端口连接回控制器上的相关输入端口
# TODO: 将wsg_source连接到站上的 "wsg.position" 输入端口

# 可视化轴（用于调试）
scenegraph = station.GetSubsystemByName("scene_graph")
AddFrameTriadIllustration(
    scene_graph=scenegraph,
    body=plant.GetBodyByName(f"{initials[0]}_body_link"),
    length=0.1,
)
AddFrameTriadIllustration(
    scene_graph=scenegraph,
    body=plant.GetBodyByName(f"{initials[1]}_body_link"),
    length=0.1,
)
AddFrameTriadIllustration(
    scene_graph=scenegraph, body=plant.GetBodyByName("body"), length=0.1
)

diagram = builder.Build()

NameError: name 'X_WO2initial' is not defined

In [ ]:
# 定义模拟器。
simulator = Simulator(diagram)
context = simulator.get_mutable_context()
station_context = station.GetMyContextFromRoot(context)
integrator.set_integral_value(
    integrator.GetMyContextFromRoot(context),
    plant.GetPositions(
        plant.GetMyContextFromRoot(context),
        plant.GetModelInstanceByName("iiwa"),
    ),
)
diagram.ForcedPublish(context)
print(f"sanity check, simulation will run for {traj_V_G.end_time()} seconds")

# 运行模拟！
meshcat.StartRecording()
if running_as_notebook:
    simulator.set_target_realtime_rate(1.0)
simulator.AdvanceTo(traj_V_G.end_time())
meshcat.StopRecording()
meshcat.PublishRecording()

In [ ]:
# 如果你想的话，使用这个来测试你的实现！
Grader.grade_output([TestPickPlacePoses], [locals()], "results.json")
Grader.print_test_results("results.json")

# 改进计算的抓取姿势

恭喜！IIWA现在应该在操纵你的首字母，使它们放置在机器人前面。现在，返回到meshcat，在右上角选择“打开控件”，然后“场景”，然后“drake”，然后“接近”。你会注意到碰撞几何目前表示为字母周围的边界框。

如果机器人无法抓住字母，或者字母似乎在机器人手指中滑动，尝试不同的抓取姿势。不同的字母可能需要更高或更低的抓取。

**一旦一切正常，拍摄视频并上传到gradescope！**